In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()

In [2]:
def Model_Eigen_WT(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True,theta_c = 0):
    if save_num == None:
        save_num = model_num
    dataset_name = 'WT'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    p.theta_c.value = theta_c
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(1,40,79))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e4]))
    vars_ls.append((p.tau_1,[5,0,-5]))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')

    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos,fp = fp)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [3]:
def Model_Eigen_HALE(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True,theta_c = 0):
    if save_num == None:
        save_num = model_num
    dataset_name = 'HALE'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 0.0889
    p.s_t.value = 16
    p.c.value = 1
    p.EI.value = 2e4
    p.GJ.value = 1e4
    p.rho_t.value = 0.75
    p.e_0.value = 0.25
    p.e_1.value = 0
    
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    p.theta_c.value = theta_c
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(1,40,79))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e4]))
    vars_ls.append((p.tau_1,[5,0,-5]))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos,fp = fp)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [5]:
def Model_Eigen_JEC(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True,theta_c = 0):
    if save_num == None:
        save_num = model_num
    
    dataset_name = 'JEC'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 12
    p.c.value = 1.8
    p.EI.value = 9.77e6
    p.GJ.value = 0.99e6
    p.rho_t.value = 19.53
    p.e_0.value = 0.08
    p.e_1.value = 0  
    p.ratio_fwt.value = 0.2
    
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    p.k_fwt.value = 0
    
    p.theta_c.value = theta_c
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,100,101))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e7]))
    vars_ls.append((p.tau_1,np.deg2rad([5,0,-5])))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    fp = [0]*p.qs
    fp[-1] = theta_c
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos,fp = fp)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [6]:
%%time
b_modes = 3
t_modes = 3

Model_Eigen_JEC(6,b_modes,t_modes)

M6-JEC: Started


/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


M6-JEC: Complete
CPU times: user 1min 23s, sys: 754 ms, total: 1min 24s
Wall time: 1min 24s


In [6]:
b_modes = 3
t_modes = 3
pool = mp.Pool(mp.cpu_count())

# running all the models
for k in range(6):
#for k in [4,5]:
    pool.apply_async(Model_Eigen_WT,args=(k,b_modes,t_modes))
    pool.apply_async(Model_Eigen_JEC,args=(k,b_modes,t_modes))
    pool.apply_async(Model_Eigen_HALE,args=(k,b_modes,t_modes))




# linearise the full model about the zero position
#pool.apply_async(Model_Eigen_WT,args=(0,b_modes,t_modes,6,True))
#pool.apply_async(Model_Eigen_JEC,args=(0,b_modes,t_modes,6,True))
#pool.apply_async(Model_Eigen_HALE,args=(0,b_modes,t_modes,6,True))

# linearise the no ZFR about the zero position
#pool.apply_async(Model_Eigen_WT,args=(1,b_modes,t_modes,7,True))
#pool.apply_async(Model_Eigen_JEC,args=(1,b_modes,t_modes,7,True))
#pool.apply_async(Model_Eigen_HALE,args=(1,b_modes,t_modes,7,True))

#fp = [0]*(b_modes+t_modes+2)
#ang = np.deg2rad([30,15,0,-10,-30])
#index = 8
#for i in range(len(ang)):
#    pool.apply_async(Model_Eigen_WT,args=(6,b_modes,t_modes,index,False,False,ang[i]))
#    pool.apply_async(Model_Eigen_JEC,args=(6,b_modes,t_modes,index,False,False,ang[i]))
#    pool.apply_async(Model_Eigen_HALE,args=(0,b_modes,t_modes,index,False,False,ang[i]))
#    index += 1 


pool.close()
pool.join()

M0-JEC: Started
M1-WT: Started
M0-HALE: Started
M1-JEC: Started
M2-JEC: Started
M0-WT: Started
M1-HALE: Started
M2-WT: Started
M2-HALE: Started
M3-WT: Started
M3-JEC: Started
M3-HALE: Started
M4-WT: Started
M4-JEC: Started
M4-HALE: Started
M5-WT: Started
M5-JEC: Started
M5-HALE: Started


<string>:117: RuntimeWarning: overflow encountered in double_scalars
<string>:150: RuntimeWarning: overflow encountered in double_scalars
<string>:118: RuntimeWarning: overflow encountered in double_scalars
<string>:151: RuntimeWarning: overflow encountered in double_scalars
<string>:103: RuntimeWarning: overflow encountered in double_scalars
<string>:115: RuntimeWarning: overflow encountered in double_scalars
<string>:115: RuntimeWarning: invalid value encountered in double_scalars
/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/_lsq/dogbox.py:199: RuntimeWarning: invalid value encountered in less
  active_set = on_bound * g < 0
Process ForkPoolWorker-5:
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent 

KeyboardInterrupt
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "<lambdifygenerated-5>", line 239, in _lambdifygenerated
    return (array([[1000000.0, -rep_100*rep_51 - rep_116*rep_51 - rep_45*rep_51 - rep_50*rep_84 - rep_51*rep_67, -rep_123*rep_51 - rep_124*rep_51 - rep_125*rep_51 - rep_126*rep_51 - rep_127*rep_51, -rep_134*rep_51 - rep_135*rep_51 - rep_136*rep_51 - rep_137*rep_51 - rep_138*rep_51, -rep_148*rep_51 - rep_151*rep_51 - rep_154*rep_51 - rep_157*rep_51 - rep_160*rep_51 - 319.205919797706*rep_49, -rep_166*rep_51 - rep_167*rep_51 - rep_168*rep_51 - rep_169*rep_51 - rep_170*rep_51 - 2042.91788670532*rep_49, -rep_171*rep_48 - rep_177*rep_51 - rep_178*rep_51 - rep_179*rep_51 - rep_180*rep_51 - rep_181*rep_51, rep_182*rep_184 + rep_184*rep_185 + rep_184*rep_186 + rep_184*rep_187 + rep_184*rep_188 - rep_194*rep_51 - rep_195*rep_51 - rep_196*rep_51 - rep_197*rep_51 - rep_198*rep_51], [0, -rep_100*rep_2

  File "<lambdifygenerated-3>", line 17, in _lambdifygenerated
    rep_13 = sin(Lambda)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/utilities/lambdify.py", line 812, in lambdify
    expr_str = str(expr)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/matrices/matrices.py", line 2408, in __str__
    return "Matrix(%s)" % str(self.tolist())
KeyboardInterrupt
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 410, in __repr__
    return sstr(self, order=None)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/str.py", line 871, in sstr
    s = p.doprint(expr)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/printer.py", line 249, in doprint
    return self._str(self._print(expr))
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/printer.py", line 287, in _print
    return getattr(self, printmethod)(expr, **kwargs)
  

  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/str.py", line 153, in _print_Function
    return expr.func.__name__ + "(%s)" % self.stringify(expr.args, ", ")
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 1719, in _accept_eval_derivative
    return s._visit_eval_derivative_scalar(self)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/str.py", line 36, in stringify
    return sep.join([self.parenthesize(item, level) for item in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/str.py", line 36, in <listcomp>
    return sep.join([self.parenthesize(item, level) for item in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 1724, in _visit_eval_derivative_scalar
    return base._eval_derivative(self)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/printing/str.py", line 33, in paren

  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify/simplify.py", line 1147, in bottom_up
    args = tuple([bottom_up(a, F, atoms, nonbasic) for a in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify/simplify.py", line 1147, in <listcomp>
    args = tuple([bottom_up(a, F, atoms, nonbasic) for a in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify/simplify.py", line 1147, in bottom_up
    args = tuple([bottom_up(a, F, atoms, nonbasic) for a in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify/simplify.py", line 1147, in <listcomp>
    args = tuple([bottom_up(a, F, atoms, nonbasic) for a in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify/simplify.py", line 1147, in bottom_up
    args = tuple([bottom_up(a, F, atoms, nonbasic) for a in args])
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/simplify

In [155]:
model_num = 0
b_modes =3
t_modes = 3
save_num = 6
linearise = False
calc_fixed_pos = True
theta_c = 0
dataset_name = 'JEC'
p = rw.base_params(b_modes + t_modes + 2)
p.rho.value = 1.225
p.s_t.value = 12
p.c.value = 1.8
p.EI.value = 9.77e6
p.GJ.value = 0.99e6
p.rho_t.value = 19.53
p.e_0.value = 0.08
p.e_1.value = 0  
p.ratio_fwt.value = 0.2

p.alpha_s.value = np.deg2rad(15)
p.eta.value = [1]*p.qs
p.m_ac.value = 1e6
p.k_ac.value = 1e6
p.k_fwt.value = 0

p.theta_c.value = theta_c

vars_ls =[]
vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
vars_ls.append((p.V,np.linspace(1,100,100))) # V must be second
vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
vars_ls.append((p.c_dmax,[0,1]))
vars_ls.append((p.m_factor,[0.5,1,1.5]))
vars_ls.append((p.k_fwt,[0,1e7]))
vars_ls.append((p.tau_1,[1,0]))

print(f'M{model_num}-{dataset_name}: Started')
sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
variables = [k for k,v in vars_ls]
if linearise:
    sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
fp = [0]*p.qs
fp[-1] = theta_c

M0-JEC: Started


In [156]:
model = sm
calc_fixed_points = True
jac=True
fixed_point_gen=None
fp=[0]*p.qs

#get list of symbols to key in model
variables = [k for k,v in vars_ls]

model_mini = model.msubs(p.GetSubs(0,p.fp,ignore=variables))
#model_lin = model_mini.linearise(p)

# get eigen Matrices and turn into a function
#K,M = model_lin.GeneralEigenProblem(p)
K,M = model_mini.GeneralEigenProblemLin(p)

#get free body problem
K_v0 = me.msubs(K,{sym.Symbol(p.V.name):0})
M_v0 = me.msubs(M,{sym.Symbol(p.V.name):0})
func = sym.lambdify((variables,p.fp),(K,M))
func_v0 = sym.lambdify((variables,p.fp),(K_v0,M_v0))

# If caluclating fixed points generate require objective functions
if calc_fixed_points:
    if fixed_point_gen is None:
        f = me.msubs((model_mini.f-model_mini.ExtForces.Q()),{i:0 for i in p.qd})
    else:
        f = fixed_point_gen(model_mini)
    f = me.msubs(f,{p.q[0]:0})
    f_v0 = me.msubs(f,{sym.Symbol(p.V.name):0})

    func_obj = sym.lambdify((p.q,variables),f,"numpy")
    func_obj_v0 = sym.lambdify((p.q,variables),f_v0,"numpy")

    if jac:
        func_jac_obj = sym.lambdify((p.q,variables),f.jacobian(p.q),"numpy")
        #func_jac_obj_v0 = sym.lambdify((p.q,variables),f_v0.jacobian(p.q),"numpy")

KeyboardInterrupt: 

In [ ]:
from scipy.optimize import fsolve,least_squares
# Get all possible combinations of the variables
perms = np.array(np.meshgrid(*[v for k,v in vars_ls ])).T.reshape(-1,len(vars_ls))

df_perms = [[(vars_ls[i][0],row[i]) for i in range(len(row))] for row in perms]
string_perms = [{vars_ls[i][0].name:row[i] for i in range(len(row))} for row in perms]


In [ ]:
%%time
ff = lambda q,v:func_obj(q,values)[:,0]
np.diag(func_jac_obj(q,values))

In [93]:
func_obj(q,values)

array([[ 3.32397859e+03],
       [ 1.63422795e+03],
       [ 8.11771410e+03],
       [-5.26943555e+02],
       [ 4.86524249e-02],
       [ 7.00590630e-01],
       [ 8.06922960e+00],
       [ 4.43631283e+01]])

In [153]:
%%time
i = 0

guess = [0]*p.qs
guess[-1] =0.1

values = tuple([v for k,v in df_perms[i]])

lb = [-np.inf]*p.qs
ub = [np.inf]*p.qs
lb[0] =-0.1
ub[0] = 0.1
lb[-1] = -2
ub[-1] = 2
bounds = (lb,ub)

q = least_squares(lambda q,v:func_obj(q,values)[:,0],guess,method='dogbox',jac=func_jac_obj, args = (values,)).x
guess = q
q

CPU times: user 5.39 ms, sys: 1.94 ms, total: 7.33 ms
Wall time: 5.64 ms


array([ 0.00000000e+00, -1.01635565e-03,  6.35264375e-05, -1.47064327e-06,
        1.02356803e-03, -5.33472701e-05,  2.81942665e-09,  1.49672153e+00])

In [154]:
%%time
root(lambda q,v: func_obj(q,v)[:,0],guess, args=(values,), method='hybr', jac=func_jac_obj).x

CPU times: user 1.13 ms, sys: 11 µs, total: 1.14 ms
Wall time: 1.14 ms


array([-1.23131824e+10, -1.01635565e-03,  6.35264375e-05, -1.47064327e-06,
        1.02356803e-03, -5.33472701e-05,  2.81942665e-09,  1.49672153e+00])

In [114]:
%%time
i = 99

guess = [0]*p.qs
guess[-1] =0.1

values = tuple([v for k,v in df_perms[i]])
q = fsolve(lambda q,v: func_obj(q,v)[:,0],guess,fprime =func_jac_obj, factor = 1,args=(values,))  
q         

CPU times: user 26.3 ms, sys: 815 µs, total: 27.1 ms
Wall time: 26.4 ms


array([-3.45874512e+161,  4.67139809e-004,  8.78151963e-006,
        4.16153153e-007, -2.72820811e-003, -5.83508559e-005,
        1.79766059e-005, -9.16830125e-001])

In [ ]:
%%time
i = 1000

guess = [0]*p.qs
guess[-1] =0.1

values = tuple([v for k,v in df_perms[i]])   
q = least_squares(lambda q,v:func_obj(q,values)[:,0],guess,method='lm',jac=func_jac_obj,args = (values,)).x
guess = q
q
#qs.append(q)
#x = [0]*p.qs*2
#x[::2] = q
#if string_perms[i]["V"] == 0:
#    evals, evecs = eig(*func_v0(values,x))
#else:
#    evals, evecs = eig(*func(values,x))
#jac_dat = mf.ExtractEigenValueData_list(evals,evecs,sortby='F')
#for j in range(len(jac_dat)):    
#        flutdfv2.append({**jac_dat[j],**string_perms[i],'q':q})



In [68]:
string_perms[99]

{'Lambda': 0.17453292519943295,
 'V': 100.0,
 'alpha_r': 0.0,
 'c_dmax': 0.0,
 'delta_m': 0.5,
 'k_fwt': 0.0,
 'tau_1': 1.0}

In [139]:
%%time
flutdfv2 = []
qs = []
#for i in range(len(df_perms)):
for i in range(1000):
    values = tuple([v for k,v in df_perms[i]])      
    # Calc fixed point
    #set the initial guess (if v=0 set to FWT dropped doen else use previous result)
    if calc_fixed_points:
        if string_perms[i]["V"] == 0:
            guess = [0]*p.qs
            guess[-1] = np.pi/2
            q = fsolve(lambda q,v: func_obj_v0(q,v)[:,0],guess,factor = 1,args=(values,))  
        else:
            guess = [0]*p.qs
            guess[-1] =0.1
            #guess = [0]*p.qs if i==0 else qs[-1]
            q = root(lambda q,v:func_obj(q,values)[:,0],guess,jac=func_jac_obj if jac else '2-point',args = (values,)).x
    else:
        q=fp

    qs.append(q)
    x = [0]*p.qs*2
    x[::2] = q
    if string_perms[i]["V"] == 0:
        evals, evecs = eig(*func_v0(values,x))
    else:
        evals, evecs = eig(*func(values,x))

    jac_dat = mf.ExtractEigenValueData_list(evals,evecs,sortby='F')
    #create q and perm data to match size
    for j in range(len(jac_dat)):    
        flutdfv2.append({**jac_dat[j],**string_perms[i],'q':q})


CPU times: user 7.31 s, sys: 19 ms, total: 7.33 s
Wall time: 7.35 s
